In [12]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR



In [14]:
# spin number
N = 10
dim_v = 2
dim_h = 10

# Model
# 假設現在只有10個spin

class Net(nn.Module):
    def __init__(self, dim_v, dim_h, N):
        '''
        放上網路裡面有什麼功能
        '''
        super(Net, self).__init__()
        self.hidden_dim = dim_h
        #nn.Linear 裡面要放 matrix W 的維度
        self.parameters = [nn.Linear(dim_h, (dim_h+dim_v))]*N
        #這個self.hidden2out的linear是接，softmax的linear，這個做完之後丟進softmax
        self.hidden2out = [nn.Linear(dim_v, dim_h)]*N
        

    def forward(self, hidden, one_hot_spin):
        '''
        放上根據上面的功能，我們的網絡是怎麼連接功能的，最後要有一個return 值
        ''' 
        # init, dimension在這邊可能不對 
        temp_linear1 = [0]*N
        temp_linear2 = [0]*N
        zero_or_one = [[0]*dim_v]*N
        spin = [0]*N
        for i in range(N):
            # Linear = W[𝜎;h]+b
            temp_linear1[i] = self.parameters[i](hidden[i] + one_hot_spin[i])
            # ELU: https://pytorch.org/docs/stable/generated/torch.nn.ELU.html
            ###!!!!!!!這裡的idx可能是錯的（滾滾滾
            hidden[i+1] = nn.ELU(temp_linear1[i])
            # 進去softmax前的linear，(Un*hn+cn), 我不確定dim_h是不是這樣餵ＱＱ
            temp_linear2[i] = self.hidden2out[i](hidden[i+1])
            # 進去softmax
            zero_or_one[i] = F.softmax(temp_linear2[i], dim = dim_h)
            # torch.mm 內積：內積mm(, spin[i])         
        return zero_or_one



In [31]:
# calculate Hamiltonian
def calculate_Ham (spin, J):
    Hamiltonain = 0
    for i in range (N-1):
        Hamiltonain += (spin[i]*spin[i+1]*J[i])
    return Hamiltonain
def Sample(zero_or_one):
    spin = [0]*N
    for i in range (N):
        spin[i] = 2*(zero_or_one[i].argmax(dim=1, keepdim=True))-1
    return spin

epochs = 5
# Train cell

def train(model, training_data, optimizer, data_num, J, ground_Ham, epoch):
    model.train()
    
    # data_num：指的是有幾組 data
    Hamiltonian = [0]*data_num
    hidden = [[0]*dim_h]*N # 怎麼那麼複雜啊（滾滾滾
    # 亂給的，要調整
    hidden[0] = [0.5]*dim_h
    running_loss = 0.0
    zero_or_one = [[0]*dim_v]*N
    
    for running_dataNum in range (data_num):
        optimizer.zero_grad() 
        zero_or_one = model.forward(hidden, one_hot_spin)
        spin = Sample(zero_or_one)
        Hamiltonian[running_dataNum] = calculate_Ham(spin, J)
        running_loss = (Hamiltonian[running_dataNum] - ground_Ham)
        running_loss.backward()
        optimizer.step()
        print('epoch = ', epoch,', running_dataNum = ', running_dataNum,', running_loss = ', running_loss)
    print('Epoch =', epoch, ' Training Finished,', ', Running_loss = ', running_loss)

In [ ]:
# Test cell
def test(model, testing_data, data_num, J, ground_Ham):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for running_dataNum in range(data_num):
            spin, hidden = mode.forward(spin, hidden, one_hot_spin)
            Hamiltonian[running_dataNum] = calculate_Ham(spin, J)
            test_loss = (Hamiltonian[running_dataNum] - ground_Ham)
            print('Testing num = ', running_dataNum, 'test_loss = ', test_loss)

In [32]:
def main():
    # readin data
    '''
    TODO
    '''
    # construct model
    model = Net(dim_v, dim_h, N)
    # optimizer
    optimizer = optim.SGD(Net.parameters(), lr=0.002, momentum=0.9)
    # StepLR
    #scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    
    for epoch in range(epochs):
        train(model, training_data, optimizer, data_num, J, ground_Ham, epoch)
        test(model, device, test_loader)
        #scheduler.step()

        
if __name__ == '__main__':
    main()

TypeError: parameters() missing 1 required positional argument: 'self'

In [30]:
# try
def trying(A, B):
    A[0]=100
    A[2]=200
    B[0] = 1212
    return A, B
    
a = [1,2,3,4,5]
b = [888, 999, 10000]
print('before: a =', a)
print('before: b =', b)
c=[]
d=[]
c,d = trying(a,b)
print('after: c = ', c)
print('after: d = ', d)

before: a = [1, 2, 3, 4, 5]
before: b = [888, 999, 10000]
after: c =  [100, 2, 200, 4, 5]
after: d =  [1212, 999, 10000]


In [33]:
spin=[1,2,3]
spin.backward()

AttributeError: 'list' object has no attribute 'backward'